In [1]:
#upload data:
import pandas as pd

# Load the CSV file
data = pd.read_csv('data_QA.csv')

# Assuming the CSV has two columns: 'question' and 'answer'
questions = data['question'].tolist()
answers = data['text'].tolist()


In [2]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from torch import cuda

embed_model_id = 'sentence-transformers/all-MiniLM-L6-v2'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

embed_model = HuggingFaceEmbeddings(
    model_name=embed_model_id,
    model_kwargs={'device': device},
    encode_kwargs={'device': device, 'batch_size': 96}
)

C:\Users\drief\AppData\Roaming\Python\Python39\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
C:\Users\drief\AppData\Roaming\Python\Python39\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [3]:
from langchain.docstore.document import Document

docs = [Document(page_content=answer, metadata={"question": question}) for question, answer in zip(questions, answers)]


In [4]:
from langchain_community.vectorstores import FAISS
# Créez l'index FAISS
db = FAISS.from_documents(docs, embed_model)

C:\Users\drief\AppData\Roaming\Python\Python39\site-packages\transformers\models\bert\modeling_bert.py:435: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


In [26]:
query = 'la faculté des sciences et techniques de tanger'#"Quelle est la dates des examens par local?"
hhh = db.similarity_search_with_score(query)
print(type(float(hhh[0][-1])))

<class 'float'>


In [13]:
def generate_answer(quest,model_name="mistral"):
    from langchain_community.chat_models import ChatOllama
    from langchain_core.output_parsers import StrOutputParser
    from langchain_core.prompts import ChatPromptTemplate

    similar_doc=db.similarity_search_with_score(quest)
    get_info=similar_doc[0][0].page_content
    llm = ChatOllama(model=model_name)
    prompt = ChatPromptTemplate.from_template("Résumer en français le text donné sous forme d'une paragraphe {topic}")
    chain = prompt | llm | StrOutputParser()
    return chain.invoke({"topic":get_info})

In [8]:
%%time
question='la faculté des sciences et techniques de tanger'
print(generate_answer(question))

 La Faculté des Sciences et Techniques de Tanger (FSTT) dépend de l'Université Abdelmalek Essaâdi et a été fondée en 1995. Elle fait partie des établissements d'enseignement supérieur réguliers et est chargée de la formation initiale dans les domaines scientifiques et techniques, de la formation continue ainsi que de la recherche et du développement dans les domaines des sciences et technologies.
CPU times: total: 266 ms
Wall time: 4min 30s


In [27]:
def generate_answer(quest, model_name="mistral"):
    from langchain_community.chat_models import ChatOllama
    from langchain_core.output_parsers import StrOutputParser
    from langchain_core.prompts import ChatPromptTemplate
    import torch
    # Liste de salutations courantes
    salutations = ["bonjour", "bonsoir", "salut", "coucou", "hello", "hi"]
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    #print(device)
    # Vérifier si la requête est une salutation
    if quest.lower().strip() in salutations:
        return "salut ! Comment puis-je vous aider aujourd'hui ?"

    # Rechercher des documents similaires
    similar_doc = db.similarity_search_with_score(quest)
    
    # Si aucun document similaire n'est trouvé, utiliser un prompt générique
    if similar_doc[0][-1] >=0.85:
        llm = ChatOllama(model=model_name, device=device)
        generic_prompt = ChatPromptTemplate.from_template("Répondre de manière appropriée à la question suivante : {question}")
        chain = generic_prompt | llm | StrOutputParser()
        return chain.invoke({"question": quest})
    
    # Si des documents similaires sont trouvés, résumer le contenu du document
    get_info = similar_doc[0][0].page_content
    llm = ChatOllama(model=model_name, device=device)
    specific_prompt = ChatPromptTemplate.from_template("Résumer en français le texte donné sous forme d'un paragraphe {topic}")
    chain = specific_prompt | llm | StrOutputParser()
    return chain.invoke({"topic": get_info})


In [12]:
#%%time
#question='la faculté des sciences et techniques de tanger'
#print(generate_answer(question))

cuda
 La Faculté des Sciences et Techniques de Tanger (FSTT), liée à l'Université Abdelmalek Essaâdi, a été fondée en 1995. Elle appartient aux établissements d'enseignement supérieur réguliers et ses missions comprennent la formation initiale dans les domaines scientifiques et techniques, ainsi que la formation continue, la recherche et le développement dans les domaines des sciences et technologies.
CPU times: total: 234 ms
Wall time: 4min


In [31]:
%%time
response = generate_answer("Information sur la faculté des sciences et technique de Tanger")
print(response)  


cuda
 La Faculté des Sciences et Techniques de Tanger (FSTT), rattachée à l'Université Abdelmalek Essaâdi, a été fondée en 1995. Elle fait partie des établissements supérieurs d'accès régulier et sa mission est la formation initiale dans les domaines scientifiques et techniques, ainsi que la formation continue et la recherche et le développement dans les domaines des sciences et techniques.
CPU times: total: 375 ms
Wall time: 3min 51s


In [34]:
from flask import Flask, request, jsonify

app = Flask(__name__)

@app.route('/process', methods=['POST'])
def process():
    data = request.get_json()
    query = data.get('query', '')
    modell = data.get('model_name','')
    print(modell)
    result = generate_answer(query,modell)
    return jsonify({'result': result})

# Run the Flask server
if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.69.128:5000
Press CTRL+C to quit
127.0.0.1 - - [05/Jun/2024 01:59:51] "POST /process HTTP/1.1" 200 -


mistral
cuda


127.0.0.1 - - [05/Jun/2024 02:00:16] "POST /process HTTP/1.1" 200 -


gemma
cuda
